# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [33]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import datetime
import time
import re

In [39]:
current_time = time.strftime("%Y/%m/%d",time.localtime())
print(current_time)
print(type(current_time))


2020/03/24
<class 'str'>


In [84]:
PATH = '/Users/lucakao/PythonWorkspace/2st-PyCrawlerMarathon/webdriver/chromedriver'
browser = webdriver.Chrome(executable_path=PATH)
browser.get("https://www.ettoday.net/news/news-list.htm")


In [66]:
# 每2秒執行一次滾動視窗的JS程式碼
for i in range(10):
    time.sleep(1)
    browser.execute_script('window.scrollTo(1,10000);')

In [30]:
# 取得資料，丟到 BeautifulSoup 解析
html = browser.page_source
soup = BeautifulSoup(html, 'html5lib')
for row in soup.find(class_= 'part_list_2').find_all('h3'):
    if row.find(class_= 'date').text.split(' ')[0] == current_time:
        date = row.find(class_= 'date').text
        title = row.find(name= 'a').text
        tag = row.find(name= 'em').text
        link = row.find(name= 'a')['href']
        print(date,tag,title,"https://www.ettoday.net"+link)

2020/03/24 10:26 國際 紐約市首名校長感染新冠肺炎病逝！　師生是否染疫待釐清 https://www.ettoday.net/news/20200324/1674975.htm
2020/03/24 10:24 大陸 Now新聞台剪接師中標！　港第二位媒體工作者確診...女友隔離中 https://www.ettoday.net/news/20200324/1675007.htm
2020/03/24 10:24 生活 館長要賣衛生紙！64包888元含運「比好市多便宜」　備20萬箱：我隨時可以賣 https://www.ettoday.net/news/20200324/1674941.htm
2020/03/24 10:21 社會 快訊／國道3號「7車連環撞」全線車道封閉！1人命危、多人輕重傷　警消急搶救 https://www.ettoday.net/news/20200324/1675003.htm
2020/03/24 10:21 影劇 「0收入餘額不足」跟爸媽借錢　《星光》男星消失唱片圈…背後原因曝光 https://www.ettoday.net/news/20200324/1674967.htm
2020/03/24 10:20 影劇 劉真生前曾認「當媽後變得很怕死」！　40歲才生下女兒…卻再無法陪她長大 https://www.ettoday.net/news/20200324/1675001.htm
2020/03/24 10:18 國際 曾使10城鎮5萬人隔離　義大利「1號病人」出院籲：請留在家裡 https://www.ettoday.net/news/20200324/1674997.htm
2020/03/24 10:18 生活 「劉真很不幸成為那1%」　醫遺憾：手術傷口越小，風險越高 https://www.ettoday.net/news/20200324/1674944.htm
2020/03/24 10:17 體育 美國聲明力促「東奧延期舉行」　洛夫支持：不知道世界會變怎樣 https://www.ettoday.net/news/20200324/1674998.htm
2020/03/24 10:15 政治 蘇貞昌：我們不必封城　自我管理好受害就會少 https://www.ettoday.net/news/20

In [83]:
# 關閉瀏覽器
browser.quit()

## 2. 取出現在時間兩小時內的新聞

In [41]:
#str -> time -> timestamp
def str_to_timestamp(str_time):
    str_tmp = str(str_time)
    time_tmp = time.strptime(str_tmp, "%Y/%m/%d %H:%M")
    timestamp = time.mktime(time_tmp)
    return timestamp

In [43]:
# 拿到的是字串－變成時戳－必須要大於 > 兩小時前的時戳 (小於就是比2小時前更早了)
two_hours_ago = float(time.time()) - 7200  # 兩小時前 = int(現在時戳秒數 - 7200秒)
now = time.strftime('%Y/%m/%d %H:%M:%S',time.localtime())
before = time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(two_hours_ago))
print('現在時間:{}，獲取兩小時前:{}之後的新聞\n'.format(now, before))

現在時間:2020/03/24 10:52:11，獲取兩小時前:2020/03/24 08:52:11之後的新聞



In [65]:
for row in soup.find(class_= 'part_list_2').find_all('h3'):
    date = row.find(class_= 'date').text
    if str_to_timestamp(date) > two_hours_ago:
        date = row.find(class_= 'date').text
        title = row.find(name= 'a').text
        tag = row.find(name= 'em').text
        link = row.find(name= 'a')['href']
        print(date,tag,title,"https://www.ettoday.net"+link)


2020/03/24 10:26 國際 紐約市首名校長感染新冠肺炎病逝！　師生是否染疫待釐清 https://www.ettoday.net/news/20200324/1674975.htm
2020/03/24 10:24 大陸 Now新聞台剪接師中標！　港第二位媒體工作者確診...女友隔離中 https://www.ettoday.net/news/20200324/1675007.htm
2020/03/24 10:24 生活 館長要賣衛生紙！64包888元含運「比好市多便宜」　備20萬箱：我隨時可以賣 https://www.ettoday.net/news/20200324/1674941.htm
2020/03/24 10:21 社會 快訊／國道3號「7車連環撞」全線車道封閉！1人命危、多人輕重傷　警消急搶救 https://www.ettoday.net/news/20200324/1675003.htm
2020/03/24 10:21 影劇 「0收入餘額不足」跟爸媽借錢　《星光》男星消失唱片圈…背後原因曝光 https://www.ettoday.net/news/20200324/1674967.htm
2020/03/24 10:20 影劇 劉真生前曾認「當媽後變得很怕死」！　40歲才生下女兒…卻再無法陪她長大 https://www.ettoday.net/news/20200324/1675001.htm
2020/03/24 10:18 國際 曾使10城鎮5萬人隔離　義大利「1號病人」出院籲：請留在家裡 https://www.ettoday.net/news/20200324/1674997.htm
2020/03/24 10:18 生活 「劉真很不幸成為那1%」　醫遺憾：手術傷口越小，風險越高 https://www.ettoday.net/news/20200324/1674944.htm
2020/03/24 10:17 體育 美國聲明力促「東奧延期舉行」　洛夫支持：不知道世界會變怎樣 https://www.ettoday.net/news/20200324/1674998.htm
2020/03/24 10:15 政治 蘇貞昌：我們不必封城　自我管理好受害就會少 https://www.ettoday.net/news/20

## 3. 根據範例，取出三天前下午三點到五點的新聞

In [51]:
today = datetime.date.today()

print(today)
print(datetime.timedelta(3))
days_3_ago = today - datetime.timedelta(3)
print(days_3_ago)

2020-03-24
3 days, 0:00:00
2020-03-21


In [121]:
# 三天前15:00的時間戳
dt = str(days_3_ago)+" 15:00"
dt=dt.replace('-', '/')
print(dt)
timeArray = time.strptime(dt, "%Y/%m/%d %H:%M")
three_clock = time.mktime(timeArray)


# 三天前17:00的時間戳
dt = str(days_3_ago)+" 17:00"
dt=dt.replace('-', '/')
print(dt)
timeArray = time.strptime(dt, "%Y/%m/%d %H:%M")
five_clock = time.mktime(timeArray)
print(three_clock)
print(five_clock)



2020/03/21 15:00
2020/03/21 17:00
1584774000.0
1584781200.0


In [118]:
# 選到三天前的日期，按下查詢按鈕
from selenium.webdriver.support.ui import Select

selectYear = Select(browser.find_element_by_id("selY"))
selectYear.select_by_value(str(timeArray.tm_year))
selectMonth = Select(browser.find_element_by_id("selM"))
selectMonth.select_by_value(str(timeArray.tm_mon))
selectDay = Select(browser.find_element_by_id('selD'))
selectDay.select_by_value(str(timeArray.tm_mday))

browser.find_element_by_css_selector('#button').click()

In [119]:
# 每2秒執行一次滾動視窗的JS程式碼
p = "."
for i in range(20):
    time.sleep(1)
    browser.execute_script('window.scrollTo(1,20000);')
    print(p+("."))

..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..


In [123]:
html = browser.page_source
soup = BeautifulSoup(html, 'html5lib')
for row in soup.find(class_= 'part_list_2').find_all('h3'):
    date = row.find(class_= 'date').text
    if five_clock >= str_to_timestamp(date) >= three_clock:
        date = row.find(class_= 'date').text
        title = row.find(name= 'a').text
        tag = row.find(name= 'em').text
        link = row.find(name= 'a')['href']
        print(date,tag,title,"https://www.ettoday.net"+link)

2020/03/21 17:00 地方 台東檢警偵破新冠肺炎不實訊息　即時遏止謠言散布 https://www.ettoday.net/news/20200321/1673328.htm
2020/03/21 17:00 旅遊 網美新聚地！特搜嘉義三大人氣IG景點　秒飛日本當櫻花妹 https://www.ettoday.net/news/20200321/1672686.htm
2020/03/21 17:00 旅遊 走進歐風小屋！台東超唯美人氣咖啡廳　招牌手沖咖啡濃厚醇香 https://www.ettoday.net/news/20200321/1670580.htm
2020/03/21 16:58 社會 南投國中生「美工刀刺死同學」送收容　父抱頭痛哭母鞠躬向被害家屬道歉 https://www.ettoday.net/news/20200321/1673326.htm
2020/03/21 16:58 新奇 「人皮客棧」真實版！他從死囚身上剝下2000張「刺青人皮」：與梅毒有關 https://www.ettoday.net/news/20200321/1673134.htm
2020/03/21 16:56 國際 獨／英國超市也淨空！直飛機票一位難求　滯倫敦台人：不敢冒險回台灣 https://www.ettoday.net/news/20200321/1672992.htm
2020/03/21 16:47 體育 UBA／林蝶抓37籃板拚到抽筋　教頭給肯定笑稱：也放槍很多 https://www.ettoday.net/news/20200321/1673327.htm
2020/03/21 16:43 生活 高雄好市多賣場直擊！民生物資充充足　貨品「多到滿出來」 https://www.ettoday.net/news/20200321/1673285.htm
2020/03/21 16:42 大陸 陸官媒反嗆陳建仁　維基百科資料做依據「舉世獨有」 https://www.ettoday.net/news/20200321/1673319.htm
2020/03/21 16:42 社會 警抓「推筒子」賭場扣27萬元　賭客要錢不要命！通通沒戴口罩 https://www.ettoday.net/news/20200321/1673293.htm
202